In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
import urllib.request
import re
from bs4 import BeautifulSoup
%pprint
%who

DATA_FOLDER: ../data/
SAVES_FOLDER: ../saves/
Pretty printing has been turned OFF
BeautifulSoup	 DATA_CSV_FOLDER	 DATA_FOLDER	 ENCODING_TYPE	 SAVES_CSV_FOLDER	 SAVES_FOLDER	 SAVES_PICKLE_FOLDER	 attempt_to_pickle	 example_iterrows	 
get_column_descriptions	 load_csv	 load_dataframes	 load_object	 math	 os	 pd	 pickle	 re	 
save_dataframes	 store_objects	 sys	 urllib	 


In [2]:

index_name = 'Technology Name'
dataframes_dict = load_dataframes(technology_availability_df='technology_availability_df')
technology_availability_df = dataframes_dict['technology_availability_df']
if 'Unnamed: 0' in technology_availability_df.columns:
    technology_availability_df = technology_availability_df.set_index('Unnamed: 0')
elif index_name in technology_availability_df.columns:
    technology_availability_df = technology_availability_df.set_index(index_name)
technology_availability_df.index.name = index_name

Attempting to load C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\pickle\technology_availability_df.pickle.
No pickle exists - attempting to load C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\csv\technology_availability_df.csv.



# Harvest the Effect column for one-hot encoding

In [3]:

technology_availability_df['Effect'].unique().tolist()

['Buildings +10% HP and +1/+1 armor', 'Castle HP +21%', 'Building LOS +4', 'Villagers build 20% faster', 'Mounted units +20 HP', 'Cavalry armor +1/+1', 'Cavalry speed +10%', 'Cavalry armor +1/+2', 'Tribute fee 0%', 'Villagers chop wood 20% faster', 'Trade Carts and Trade Cogs move 50% faster', "See allies' LOS", 'Tribute fee 20%', 'Farms +175 food', 'Fishing Ships work 25% faster', 'Villagers mine gold 15% faster', 'Villager speed +10%, carrying capacity +50%', 'Farms +125 food, Villagers +1 food carry capacity', 'Farms +75 food', 'Villagers chop wood 10% faster', 'Villager speed +10%, carrying capacity +25%', 'Huskarls can be trained at Barracks', 'Infantry +2 attack vs buildings', 'Infantry/cavalry attack +2', 'Infantry armor +1/+1', 'Infantry/cavalry attack +1', 'Infantry armor +1/+2', 'Infantry speed +10%', 'Infantry +2 LOS', 'Advance to Castle Age', 'Unit creation speed at the Barracks, Archery Range, Stable, and Castle +33%', 'Advance to Feudal Age', 'Advance to Imperial Age', 'V


# Fill in the missing Effect data by web scraping

In [ ]:

wiki_url = 'https://ageofempires.fandom.com'
def get_link_html(link):
    link_str = re.sub(' +', '_', link)
    link_url = '{}/wiki/{}'.format(wiki_url, link_str)
    try:
        extended_link_url = '{}_(Age_of_Empires_II)'.format(link_url)
        with urllib.request.urlopen(extended_link_url) as response:
            link_html = response.read()
        link_url = extended_link_url
    except:
        try:
            with urllib.request.urlopen(link_url) as response:
                link_html = response.read()
        except Exception as e:
            print('{} got this error: {}'.format(link_url, e))
            link_html = None
    
    return link_url, link_html

In [ ]:

def get_page_soup(tech_name):
    page_url, page_html = get_link_html(tech_name)
    page_soup = BeautifulSoup(page_html, 'html.parser')
    
    return page_soup

In [ ]:

def get_content_text_div(tech_name):
    page_soup = get_page_soup(tech_name)
    content_text_selector = '#mw-content-text'
    content_text_div = page_soup.select(content_text_selector)[0]
    
    return content_text_div

In [ ]:

def get_effect_from_aside(tech_soup):
    effect_value = None
    aside_list = tech_soup.select('aside')
    for aside_soup in aside_list:
        for h2 in aside_soup.find_all('h2'):
            if h2.text.strip() == 'Effect':
                h2_siblings_list = list(h2.next_siblings)
                if len(h2_siblings_list):
                    value_list = list(h2_siblings_list[1].strings)
                    effect_value = ' '.join(x.strip() for x in value_list).strip()
    
    return effect_value

In [ ]:

mask_series = (technology_availability_df['Effect'].isnull())
mask_series = mask_series | (technology_availability_df['Effect'] == '')
tech_name_list = technology_availability_df[mask_series].index.tolist()
for tech_name in tech_name_list:
    unit_soup = get_content_text_div(tech_name)
    effect_value = get_effect_from_aside(unit_soup)
    if effect_value is not None:
        technology_availability_df.loc[tech_name, 'Effect'] = effect_value


# Turn the Building column into one-hot encoding

In [ ]:

technology_availability_df.columns.tolist()

In [ ]:

temp_df = pd.get_dummies(technology_availability_df, prefix='Researched_in', columns=['Building'])
temp_df.columns = [re.sub(' +', '_', cn) for cn in temp_df.columns]
print(temp_df.columns.tolist())
temp_df.sample(5).T.tail(15)

In [ ]:

columns_list = ['Age', 'Researched_in_Archery_Range', 'Researched_in_Barracks',
                'Researched_in_Blacksmith', 'Researched_in_Castle', 'Researched_in_Dock',
                'Researched_in_Lumber_Camp', 'Researched_in_Market', 'Researched_in_Mill',
                'Researched_in_Mining_Camp', 'Researched_in_Monastery',
                'Researched_in_Siege_Workshop', 'Researched_in_Stable', 'Researched_in_Town_Center',
                'Researched_in_University', 'Cost', 'Wood', 'Stone', 'Gold', 'Food', 'Effect',
                'Heading', 'Previous', 'Intro', 'Time', 'Required', 'Next', 'Aztecs', 'Berbers',
                'Britons', 'Burmese', 'Byzantines', 'Celts', 'Chinese', 'Ethiopians', 'Franks',
                'Goths', 'Huns', 'Incas', 'Indians', 'Italians', 'Japanese', 'Khmer', 'Koreans',
                'Magyars', 'Malay', 'Malians', 'Mayans', 'Mongols', 'Persians', 'Portuguese',
                'Saracens', 'Slavs', 'Spanish', 'Teutons', 'Turks', 'Vietnamese', 'Vikings']
technology_availability_df = temp_df[columns_list]


# Fill in the missing Age and Building data by web scraping

In [ ]:

def get_age_and_building_from_aside(unit_soup):
    age_value = None
    building_value = None
    aside_list = unit_soup.select('aside')
    for aside_soup in aside_list:
        for data_source_div in aside_soup.find_all('div', {'data-source': True}):
            data_source_h3_list = data_source_div.find_all('h3')
            if len(data_source_h3_list):
                h3_siblings_list = list(data_source_h3_list[0].next_siblings)
                if len(h3_siblings_list):
                    column_name = data_source_div['data-source']
                    value_list = list(h3_siblings_list[1].strings)
                    column_value = ' '.join(x.strip() for x in value_list).strip()
                    if column_name == 'Age':
                        age_value = column_value
                    elif column_name == 'Building':
                        building_value = column_value
    
    return age_value, building_value

In [ ]:

mask_series = (technology_availability_df['Age'].isnull())
mask_series = mask_series | (technology_availability_df['Building'].isnull())
tech_name_list = technology_availability_df[mask_series].index.tolist()
for tech_name in tech_name_list:
    unit_soup = get_content_text_div(tech_name)
    age_value, building_value = get_age_and_building_from_aside(unit_soup)
    technology_availability_df.loc[tech_name, 'Age'] = age_value
    technology_availability_df.loc[tech_name, 'Building'] = building_value

In [ ]:

mask_series = (technology_availability_df.index.isin(tech_name_list))
technology_availability_df[mask_series].T.head(5).T.sample(5)


# Add in the extra techs not mentioned in the technology page

In [ ]:

space_regex = re.compile(r'\s+')
def process_small_list_of_units(appl_civ_text, row_dict):
    appl_civ_list = space_regex.split(appl_civ_text)
    for civ_name in civ_name_list:
        row_dict[civ_name] = 0
    for civ_name in appl_civ_list:
        if civ_name in civ_name_list:
            row_dict[civ_name] = 1
    
    return row_dict

In [ ]:

def process_all_except(appl_civ_text, row_dict):
    for civ_name in civ_name_list:
        if civ_name in appl_civ_text:
            row_dict[civ_name] = 0
        else:
            row_dict[civ_name] = 1
    
    return row_dict

In [ ]:

def process_exceptions(link_soup, row_dict):
    desc_p_list = list(link_soup.select('aside')[0].next_siblings)
    if len(desc_p_list):
        for sib in desc_p_list:
            if re.search('available to (all|most) civilizations,? except', str(sib)):
                break
        process_all_except(sib.text.strip(), row_dict)
    
    return row_dict

In [ ]:

def process_article(art_table_list, row_dict):
    art_table = art_table_list[0]
    art_ul_list = art_table.find_all('ul')
    assert len(art_ul_list) == 2

    # Get available civs
    available_ul = art_ul_list[0]
    for link in available_ul.find_all('a'):
        civ_name = link['title'].split('(')[0].strip()
        row_dict[civ_name] = 1

    # Get unavailable civs
    unavailable_ul = art_ul_list[1]
    for link in unavailable_ul.find_all('a'):
        civ_name = link['title'].split('(')[0].strip()
        row_dict[civ_name] = 0

    return row_dict

In [ ]:

article_selector = '#mw-content-text > table.article-table'
civ_name_list = load_object('civ_name_list')
def process_unit(link_soup, appl_civ_text, row_dict):
    for column_name in civ_name_list:
        columns_set.add(column_name)
    if 'All civilizations' in appl_civ_text:
        for civ_name in civ_name_list:
            row_dict[civ_name] = 1
    elif 'All except ' in appl_civ_text:
        row_dict = process_all_except(appl_civ_text, row_dict)
    elif 'See ' in appl_civ_text:
        art_table_list = link_soup.select(article_selector)
        if len(art_table_list):
            row_dict = process_article(art_table_list, row_dict)
        else:
            row_dict = process_exceptions(link_soup, row_dict)
    else:
        row_dict = process_small_list_of_units(appl_civ_text, row_dict)
    
    return row_dict

In [ ]:

def consume_aside(unit_soup, row_dict):
    aside_list = unit_soup.select('aside')
    for aside_soup in aside_list:
        for data_source_div in aside_soup.find_all('div', {'data-source': True}):
            data_source_h3_list = data_source_div.find_all('h3')
            if len(data_source_h3_list):
                h3_siblings_list = list(data_source_h3_list[0].next_siblings)
                if len(h3_siblings_list):
                    column_name = data_source_div['data-source']
                    value_list = list(h3_siblings_list[1].strings)
                    column_value = ' '.join(x.strip() for x in value_list).strip()
                    if column_name == 'Available':
                        row_dict = process_unit(unit_soup, column_value, row_dict)
                    else:
                        columns_set.add(column_name)
                        row_dict[column_name] = column_value
    
    return row_dict

In [ ]:

extra_tech_list = ['Bombard Cannon', 'Cannon Galleon', 'Bombard Tower']
url_list = ['{} (technology)'.format(unit) for unit in extra_tech_list]
url_list = [re.sub(' +', '_', unit) for unit in url_list]
url_list = ['{}/wiki/{}'.format(wiki_url, unit) for unit in url_list]

rows_list = []
index_list = []
columns_set = set(technology_availability_df.columns)
for technology_name in extra_tech_list:
    content_text_div = get_content_text_div(technology_name)
    if technology_name not in index_list:
        index_list.append(technology_name)
        rows_list.append(consume_aside(content_text_div, {'technology_url': page_url}))

In [ ]:

new_tech_df = pd.DataFrame(data=rows_list, index=index_list, columns=columns_set)

In [ ]:

set.symmetric_difference(set(new_tech_df.columns), set(technology_availability_df.columns))

In [ ]:

f = lambda x: int(re.sub(',', '', str(x)))
str_columns_list = ['Next', 'Cost', 'Effect', 'Heading', 'Previous']
for str_column in str_columns_list:
    new_tech_df[str_column] = new_tech_df[str_column].fillna(value='')
int_columns_list = ['Gold', 'Stone', 'Wood']
for int_column in int_columns_list:
    new_tech_df[int_column] = new_tech_df[int_column].fillna(value='0')
    new_tech_df[int_column] = new_tech_df[int_column].map(f)

In [ ]:

columns_list = new_tech_df.columns[new_tech_df.isnull().any(0)]
print(columns_list)
new_tech_df[columns_list].T

In [ ]:

new_tech_df = technology_availability_df.append(new_tech_df, sort=True)

In [ ]:

columns_list = new_tech_df.columns[new_tech_df.isnull().any(0)]
for column_name in columns_list:
    print(column_name)
    print(new_tech_df[column_name].unique())
    new_tech_df[column_name] = new_tech_df[column_name].fillna('')

In [ ]:

new_tech_df = new_tech_df.sort_index()

In [ ]:

def f(x):
    return_value = ''
    if x in ['IV', 'Imperial Age']:
        return_value = 'Imperial'
    elif x == 'III':
        return_value = 'Castle'
    elif x == 'II':
        return_value = 'Feudal'
    elif x == 'I':
        return_value = 'Dark'
    
    return return_value

new_tech_df['Age'] = new_tech_df['Age'].map(f)

In [ ]:

index_name = 'Technology Name'
new_tech_df.index.name = index_name
columns_list = technology_availability_df.columns
file_path = os.path.join(DATA_CSV_FOLDER, 'technology_availability_df.csv')
print('Saving to {}'.format(os.path.abspath(file_path)))
new_tech_df[columns_list].to_csv(file_path)
save_dataframes(include_index=True,
                technology_availability_df=new_tech_df[columns_list])
xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
file_path = os.path.join(xlsx_folder, 'technology_availability_df.xlsx')
print('Saving to {}'.format(os.path.abspath(file_path)))
new_tech_df[columns_list].to_excel(file_path)


# Fix integer columns

In [ ]:

def get_mixed(x):
    x = str(x)
    if re.search(r'([+.\d]+\s+[^+.\d]+|[^+.\d]+\s+[+.\d]+)', x):
        
        return True
    
    else:
        
        return False

digit_df = technology_availability_df.applymap(get_mixed)
for column_name in digit_df.columns[digit_df.any(axis=0)]:
    print(column_name)
    print(technology_availability_df[column_name].unique().tolist())

In [ ]:

technology_availability_df.loc['Spies', 'Gold'] = '30000'

In [ ]:

f = lambda x: int(re.sub(',', '', str(x)))
for cn in ['Wood', 'Stone', 'Gold', 'Food']:
    technology_availability_df[cn] = technology_availability_df[cn].fillna(value='0')
    technology_availability_df[cn] = technology_availability_df[cn].map(f)


# Save everything

In [ ]:

columns_list = technology_availability_df.columns
file_path = os.path.join(DATA_CSV_FOLDER, 'technology_availability_df.csv')
print('Saving to {}'.format(os.path.abspath(file_path)))
technology_availability_df[columns_list].to_csv(file_path)
save_dataframes(include_index=True,
                technology_availability_df=technology_availability_df[columns_list])
xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
file_path = os.path.join(xlsx_folder, 'technology_availability_df.xlsx')
print('Saving to {}'.format(os.path.abspath(file_path)))
technology_availability_df[columns_list].to_excel(file_path)